In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Scraping NST Using BS4 and HTML Parsing

In [2]:
url = f"https://www.nst.com.my/news/politics?page=0"
webpage = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'}).text
soup = BeautifulSoup(webpage, 'html5lib')
print(soup)

<!DOCTYPE html>
<html lang="en"><head>
<link crossorigin="" href="https://ad.crwdcntrl.net" rel="dns-prefetch"/>
<link crossorigin="" href="https//www.googletagservices.com" rel="dns-prefetch"/>
<link crossorigin="" href="https//www.googletagmanager.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://fonts.googleapis.com" rel="dns-prefetch"/>
<link href="https://tags.crwdcntrl.net" rel="preconnect"/>
<link href="https://bcp.crwdcntrl.net" rel="preconnect"/>
<link href="https://tags.crwdcntrl.net" rel="dns-prefetch"/>
<link href="https://bcp.crwdcntrl.net" rel="dns-prefetch"/>
<link as="script" href="https://assets.nst.com.my/assets/js/desktop/app.js?id=28306d5318ca708bf76f" rel="preload"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="pe9PwamwzcUfbL5ZxXoPwBtlO3jQxyrC7YsA5Hx2" name="csrf-token"/>
<meta content="politic news" name="keywords"/>
<title>Malaysia Political News | New Straits Times (NST Online)</title>
<m

In [3]:
page = -1
max_page = 943
total_info = []
total_articles = 0

while page != max_page:
    page += 1
    url = f"https://www.nst.com.my/news/politics?page={page}"
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, "html.parser")
    print(f'Printing {page}')   
    for article in soup.find_all('div',class_='content pl-2 pl-lg-3 col'):
        total_articles += 1
        title = article.find('h6',class_='field-title').text #get article titles
        description = article.find('div',class_='d-block article-teaser').text #get article descs
        category = article.find('span',class_='field-category').text
        time = article.find('span',class_='created-ago').text
        row_info = [title,description,time,category]
        total_info.append(row_info)  
   
print(f'Total Articles : {total_articles}')
 

Printing 0
Printing 1


KeyboardInterrupt: 

In [104]:
df = pd.DataFrame(total_info,columns=['title','description','publisheddate','section','site'])
df

,title,description,publisheddate,section,site
0,The $100bn ghost city in southern Malaysia,The Chinese-built city was supposed to be a th...,15 hours ago,News,Asia
1,Snoop Dogg & an ex-Wolves midfielder - meet Ma...,Former Wolves midfielder Hong Wan has moved fr...,2 days ago,Sport,Football
2,Forest City: Inside Malaysia's Chinese-built '...,"Built during the Chinese property boom, Forest...",6 days ago,News,Business
3,Malaysian killer released from Australian immi...,An ex-policeman convicted in a politically cha...,13 November,News,Australia
4,MotoGP: Enea Bastianini claims first race win ...,Ducati's Enea Bastianini claims his first race...,12 November,Sport,Motorsport
...,...,...,...,...,...
285,Najib Razak: Malaysian ex-PM gets 12-year jail...,Najib Razak gets 12 years in jail after he is ...,28 July 2020,News,Asia
286,Newsday: Guilty verdict for former Malaysian p...,"Live news, business and sport from around the ...",28 July 2020,Programmes,BBC World Service
287,Newsday: Former Malaysian PM guilty of abuse o...,"Live news, business and sport from around the ...",28 July 2020,Programmes,BBC World Service
288,North Korea v Malaysia: Asian Cup qualifier is...,"""Geo-political tension"" and ""safety and securi...",17 May 2017,Sport,Football


In [105]:
df.to_csv('BBCNews.csv',index=False)